In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

/usr/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Loading Data

In [2]:
%%time
data_train=pd.read_table('rsc15_train_tr.txt')
data_val=pd.read_table('rsc15_train_valid.txt')
data_test=pd.read_table('rsc15_test.txt')

CPU times: user 8.18 s, sys: 468 ms, total: 8.65 s
Wall time: 8.64 s


In [3]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31579006 entries, 0 to 31579005
Data columns (total 3 columns):
SessionId    int64
ItemId       int64
Time         float64
dtypes: float64(1), int64(2)
memory usage: 722.8 MB


In [4]:
data_train.head()

,SessionId,ItemId,Time
0,1,214536502,1.396864e+09
1,1,214536500,1.396864e+09
2,1,214536506,1.396864e+09
3,1,214577561,1.396865e+09
4,2,214662742,1.396875e+09


In [5]:
data_val.head()

,SessionId,ItemId,Time
0,11264996,214859872,1.411911e+09
1,11264996,214859870,1.411911e+09
2,11264996,214859902,1.411911e+09
3,11264987,214714807,1.411912e+09
4,11264987,214714807,1.411912e+09


In [6]:
data_test.head()

,SessionId,ItemId,Time
0,11265009,214586805,1.412000e+09
1,11265009,214509260,1.412000e+09
2,11265017,214857547,1.412014e+09
3,11265017,214857268,1.412014e+09
4,11265017,214857260,1.412014e+09


# Create Sampled Dataset

In [14]:
data_train[:1000].to_csv('data_train.csv', index=False)

# Draft


In [7]:
data=data_train[:1000].copy()

In [8]:
# sort values
data.sort_values(['SessionId', 'Time'], inplace=True)

In [9]:
data['SessionId'].nunique()+1

264

In [10]:
len(data['SessionId'].unique())

263

In [11]:
offset_sessions = np.zeros(data['SessionId'].nunique()+1, dtype=np.int32)

In [12]:
offset_sessions

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
      dtype=int32)

In [13]:
data.groupby('SessionId').size()

SessionId
1       4
2       6
3       3
4       2
6       2
7       2
8       2
9       3
11     12
12      2
13      3
14      3
16      3
17      2
18      2
19      4
21      6
22     16
23      4
24      4
26      4
27     10
28      3
29      2
31      2
32      2
33     16
34      2
36      2
37      2
       ..
304     3
306     2
307     4
308     3
309     3
311     3
312     2
313     2
314     5
316     2
317     2
318     2
319     3
321     3
322     2
323     2
324    13
327     4
336     2
337     7
338     2
339     2
342     3
343     2
344     3
346     2
347     2
348     2
349     5
351     4
Length: 263, dtype: int64

In [14]:
data.groupby('SessionId').size().cumsum()

SessionId
1         4
2        10
3        13
4        15
6        17
7        19
8        21
9        24
11       36
12       38
13       41
14       44
16       47
17       49
18       51
19       55
21       61
22       77
23       81
24       85
26       89
27       99
28      102
29      104
31      106
32      108
33      124
34      126
36      128
37      130
       ... 
304     907
306     909
307     913
308     916
309     919
311     922
312     924
313     926
314     931
316     933
317     935
318     937
319     940
321     943
322     945
323     947
324     960
327     964
336     966
337     973
338     975
339     977
342     980
343     982
344     985
346     987
347     989
348     991
349     996
351    1000
Length: 263, dtype: int64

In [15]:
# nunique+1 seems to make a space for name or what according to next line    
offset_sessions = np.zeros(data['SessionId'].nunique()+1, dtype=np.int32)
# cumsum seems to count number of time_steps for each session.
offset_sessions[1:] = data.groupby('SessionId').size().cumsum()

In [16]:
offset_sessions

array([   0,    4,   10,   13,   15,   17,   19,   21,   24,   36,   38,
         41,   44,   47,   49,   51,   55,   61,   77,   81,   85,   89,
         99,  102,  104,  106,  108,  124,  126,  128,  130,  133,  135,
        140,  142,  144,  152,  158,  161,  165,  167,  172,  174,  178,
        184,  189,  191,  193,  195,  200,  210,  213,  216,  219,  221,
        223,  225,  227,  231,  233,  235,  238,  246,  255,  257,  262,
        266,  277,  304,  306,  308,  310,  313,  317,  319,  321,  323,
        325,  330,  332,  334,  336,  340,  342,  344,  347,  365,  367,
        369,  372,  374,  376,  380,  382,  384,  404,  406,  414,  416,
        419,  421,  423,  429,  432,  434,  436,  440,  442,  446,  453,
        455,  457,  459,  462,  470,  474,  476,  479,  481,  485,  490,
        492,  494,  500,  502,  504,  506,  511,  514,  519,  521,  524,
        526,  529,  532,  534,  536,  538,  540,  543,  547,  549,  551,
        553,  556,  558,  565,  567,  570,  573,  5

In [20]:
data.ItemId.values.shape

(1000,)

In [33]:
A=data.ItemId.values[[1,10,100]]

In [34]:
B=data.ItemId.values[[9,99,200]]

In [43]:
X = tf.placeholder(tf.int32, [3], name='input')
Y = tf.placeholder(tf.int32, [3], name='output')
state = [tf.placeholder(tf.float32, [self.batch_size, self.rnn_size], name='rnn_state') for _ in xrange(self.layers)]


feed_dict={X: A, Y: B}

In [44]:
feed_dict

{<tf.Tensor 'input_1:0' shape=(3,) dtype=int32>: array([214536500, 214716935, 214836789]),
 <tf.Tensor 'output_1:0' shape=(3,) dtype=int32>: array([214551617, 214836789, 214714794])}

In [47]:
np.arange(3)

array([0, 1, 2])

In [97]:
t